# Lending Club Loan Data 

In [8]:
import pandas as pd
import numpy as np
import pandas_profiling
import warnings
import pandas_profiling

warnings.filterwarnings('ignore')

%matplotlib inline

## Part 1. Data Exploration and Evaluation

In this section, we 

### Reading the data

In [3]:
# loading the data from csv file
data = pd.read_csv('/Users/christina/Desktop/LC/data/loan.csv', low_memory=False)

/Users/christina/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data.shape

(2260668, 145)

### Ensuring Data Types

In [52]:
data.dtypes.value_counts()

float64    105
object      36
int64        4
dtype: int64

The dataset includes some numerical and some categorical variables. Let's group them.

In [53]:
df = pd.DataFrame(data.dtypes, columns=['dtype'])
df[df.dtype==np.int64]

,dtype
loan_amnt,int64
funded_amnt,int64
revol_bal,int64
policy_code,int64


In [55]:
df[df.dtype==np.float64]

,dtype
id,float64
member_id,float64
funded_amnt_inv,float64
int_rate,float64
installment,float64
annual_inc,float64
url,float64
dti,float64
delinq_2yrs,float64
inq_last_6mths,float64


In [56]:
df[df.dtype==np.object]

,dtype
term,object
grade,object
sub_grade,object
emp_title,object
emp_length,object
home_ownership,object
verification_status,object
issue_d,object
loan_status,object
pymnt_plan,object


In [39]:
list(data.columns)[19]

'desc'

In [38]:
data.url.dtype

dtype('float64')

In [36]:
# grouping variables based on existing datatypes
numerical = []
categorical = []
integers = []

for col in list(data.columns):
    if data[col].dtype == np.float64:
        numerical.append(col)
    elif data[col].dtype == np.object:
        categorical.append(col) 
    elif data[col].dtype == np.int64:
        categorical.append(col)  

In [41]:
integers

[]

In [40]:
numerical

['id',
 'member_id',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'url',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_util',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin

In [29]:
categorical

['loan_amnt',
 'funded_amnt',
 'term',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_bal',
 'initial_list_status',
 'last_pymnt_d',
 'next_pymnt_d',
 'last_credit_pull_d',
 'policy_code',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_loan_status',
 'disbursement_method',
 'debt_settlement_flag',
 'debt_settlement_flag_date',
 'settlement_status',
 'settlement_date']